In [10]:
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
    # model="gpt-4o-mini",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "question": "올드보이",
        "answer": "🍜🔨🤯"
    }
]

# 프롬프트 예제 작성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

# 예제 프롬프트와 예제 답변을 묶어서 프롬프트로 작성
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# system 및 질문을 작성하고, 위에 작성된 예제를 부여하여 답변을 유도
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세상 모든 영화의 정보를 알고 있는 영화광입니다. 당신은 질문으로 영화 제목이 제시되면, 주어진 예제와 같이 영화의 내용을 3개의 이모지로 표현합니다. 이모지는 영화의 내용을 나타내면서도 매우 재밌습니다."),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human", "{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain('메멘토')
invoke_chain('기생충')
invoke_chain('인터스텔라')

content='⏰❓🔍'
content='🏠🐜💰'
content='🚀⏳🌌'


In [11]:
invoke_chain('처음에 물어본 영화 제목이 뭐지?')

content='메멘토였어요!'
